In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import healpy as hp
from pathlib import Path
import xarray as xr
import easygems.healpix as egh
import pandas as pd

import imageio
import os

import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [6]:
# Define the path to the Zarr dataset
# Open the Zarr store using xarray

zoom_levels = [3,4,5,6,7,8,9,10]
temp_res = 1

# UK MODEL ################################
zoom = 3; path = '/g/data/qx55/uk_node/glm.n2560_RAL3p3/data.healpix.PT%iH.z%i.zarr' % (temp_res, zoom)
ds_3 = xr.open_zarr(path)

zoom = 4; path = '/g/data/qx55/uk_node/glm.n2560_RAL3p3/data.healpix.PT%iH.z%i.zarr' % (temp_res, zoom)
ds_4 = xr.open_zarr(path)

zoom = 5; path = '/g/data/qx55/uk_node/glm.n2560_RAL3p3/data.healpix.PT%iH.z%i.zarr' % (temp_res, zoom)
ds_5 = xr.open_zarr(path)

zoom = 6; path = '/g/data/qx55/uk_node/glm.n2560_RAL3p3/data.healpix.PT%iH.z%i.zarr' % (temp_res, zoom)
ds_6 = xr.open_zarr(path)

zoom = 7; path = '/g/data/qx55/uk_node/glm.n2560_RAL3p3/data.healpix.PT%iH.z%i.zarr' % (temp_res, zoom)
ds_7 = xr.open_zarr(path)

zoom = 8; path = '/g/data/qx55/uk_node/glm.n2560_RAL3p3/data.healpix.PT%iH.z%i.zarr' % (temp_res, zoom)
ds_8 = xr.open_zarr(path)

zoom = 9; path = '/g/data/qx55/uk_node/glm.n2560_RAL3p3/data.healpix.PT%iH.z%i.zarr' % (temp_res, zoom)
ds_9 = xr.open_zarr(path)

zoom = 10; path = '/g/data/qx55/uk_node/glm.n2560_RAL3p3/data.healpix.PT%iH.z%i.zarr' % (temp_res, zoom)
ds_10 = xr.open_zarr(path)


# GERMAN MODEL ################################


zoom = 10; path = '/g/data/qx55/germany_node/d3hp003.zarr/PT1H_point_z%i_atm.zarr' % zoom
ds_10_g = xr.open_zarr(path)



In [64]:
# Set extent for sydney and fiji
lon_min = 144.690354
lon_max = 180
lat_min = -36.248889
lat_max = -13.643936


# Set extent for maritime continent
#shift_val = 360
lon_min = 128.132840
lon_max = -155.331704
lat_min = -39.753870
lat_max = 4.179808

In [59]:
# Subset to our time range of interest
start_date = "2020-03-01"
finish_date = "2021-02-28"

In [65]:
pr = ds_10.pr * 60*60 # kg m-2 s-1 to mm / hr, not subsetted
time_global = pr.sel(time=slice(start_date, finish_date))
daily_total = time_global.resample(time='1D').sum()
daily_values = daily_total.where(daily_total > 0.2)
daily_mask = (daily_total > 0.2).astype(int)
print(daily_mask)


INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: method = None, returning None
INFO:flox:_choose_engine: Choosing 'flox'
INFO:flox:find_group_cohorts: cohorts is preferred, chunking is perfect.
INFO:flox:_choose_method: method is None
INFO:flox:_choose_method: choosing preferred_method=cohorts
INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: reindex is False


<xarray.DataArray 'pr' (time: 365, cell: 12582912)> Size: 37GB
dask.array<astype, shape=(365, 12582912), dtype=int64, chunksize=(1, 1048576), chunktype=numpy.ndarray>
Coordinates:
  * cell     (cell) int64 101MB 0 1 2 3 ... 12582908 12582909 12582910 12582911
    crs      float64 8B ...
  * time     (time) datetime64[ns] 3kB 2020-03-01 2020-03-02 ... 2021-02-28


In [ ]:
frames = range(1,365)

for i in frames:
    # Get data
    data = daily_values.isel(time=i)

    # Start a new figure with Cartopy projection
    central_longitude = 180
    projection = ccrs.PlateCarree(central_longitude=central_longitude)
    fig, ax = plt.subplots(figsize=(10, 5), subplot_kw={'projection': projection})

    vmin = 0
    vmax = 350
    colours = [(1, 1, 1), (0, 0, 1)]  # White to blue
    cmap_name = 'white_blue'
    custom_cmap = mcolors.LinearSegmentedColormap.from_list(cmap_name, colours, N=256)
    ax.set_global()
    custom_cmap = 'jet'
    im = egh.healpix_show(data.values, ax=ax, cmap=custom_cmap, vmin=vmin, vmax=vmax)
    ax.coastlines()

    ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

    # Coordinates for Sydney and Fiji
    sydney_lon, sydney_lat = 151.2093, -33.8688
    fiji_lon, fiji_lat = 178.0650, -17.7134

    # Plot points
    ax.plot(sydney_lon, sydney_lat, 'ro', markersize=5, label='Sydney', transform=ccrs.PlateCarree())
    ax.plot(fiji_lon, fiji_lat, 'ro', markersize=5, label='Fiji', transform=ccrs.PlateCarree())

    # Optionally add labels
    ax.text(sydney_lon - 1, sydney_lat + 1, 'Sydney', color='red', fontsize=10, transform=ccrs.PlateCarree())
    ax.text(fiji_lon - 1, fiji_lat + 1, 'Fiji', color='red', fontsize=10, transform=ccrs.PlateCarree())

    # Add colorbar
    cbar = fig.colorbar(im, ax=ax, orientation='vertical', shrink=0.7)
    cbar.set_label("Daily Precipitation (mm)")

    timestamp = pd.to_datetime(daily_mask.time[i].values)
    ax.set_title(f"Daily Precipitation on {timestamp:%Y-%m-%d}")
    fname = f"hk25-AusNode-ExtremePrecipitation/Plots/Fiji_gif_frames/frame_{i:04d}.png"
    plt.savefig(fname, dpi=150, bbox_inches='tight')
    plt.close()


In [57]:
# Create the animation
with imageio.get_writer("hk25-AusNode-ExtremePrecipitation/Plots/Precip_animation_UMz10_greater.gif", mode="I", duration=0.5) as writer:
    for i in frames:
        fname = f"hk25-AusNode-ExtremePrecipitation/Plots/Fiji_gif_frames/frame_{i:04d}.png"
        image = imageio.imread(fname)
        writer.append_data(image)

/jobfs/141074340.gadi-pbs/ipykernel_426550/2706900582.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(fname)
